In [2]:
import numpy as np
from math import factorial
import matplotlib.pyplot as plt
from matplotlib import animation
from numpy.linalg import norm

## Eigenstates of the Harmonic oscillator

$$
|\psi_n(x)\rangle = \frac{1}{\pi 2^nn!} e^{-\frac{(x-x_0)^2}{2}} H_n\left(x-x_0 \right)
$$

where $H_n\left(x \right)$ are the Hermite polynomials of order $n$

NOTE: These are the eigenstates of the following Hamiltonian

$$
H=\frac{p^{2}}{2}+ \frac{1}{2}(x-x_{0})^{2}
$$

Any translation of $|\psi_n(x)\rangle \longrightarrow |\psi_n(x+\delta x)\rangle$ results in a state which is NO MORE an eigenstate of the original Hamiltonian.
We could then decompose $|\psi_n(x+\delta x)\rangle = \sum_{m}C_{m}|\psi_n(x)\rangle$ from some coefficients $C_{m}$ associated to the old eigenstates.

In [ ]:
import numpy as np
import math
import debugger as db
from scipy.special import factorial

def hermite(x, n):
    """
    hermite:
        Computes the Hermite polynomial of order 'n' over the real space grid 'x'.

    Parameters
    ----------
    x : np.ndarray
        Real space grid as a 1D numpy array.
    n : int
        Order of the Hermite polynomial. Must be a non-negative integer (n >= 0).

    Returns
    -------
    herm_pol : np.ndarray
        Hermite polynomial of order 'n', evaluated over the input grid 'x'.

    Pre-condition
    -------------
    - n >= 0. If n < 0, the function triggers a debugging checkpoint and halts execution.
    """
    # Pre-condition: n >= 0
    if n < 0:
        db.checkpoint(debug=True, msg=f"The order of the Hermite polynomial is not valid (n={n}, expected n>=0)", stop=True)

    # Coefficients set to 0 except for the one of order n.
    herm_coeffs = np.zeros(n + 1)
    herm_coeffs[n] = 1

    # Actual computation of the polynomial over the space grid.
    herm_pol = np.polynomial.hermite.hermval(x, herm_coeffs)
    return herm_pol


def stationary_state(x, omega, n):
    """
    harmonic_wfc:
        Computes the wavefunction of order 'n' for a quantum harmonic oscillator, 
        defined over the real space grid 'x'.

    Parameters
    ----------
    x : np.ndarray
        Real space grid as a 1D numpy array.
    omega : float
        Angular frequency of the harmonic oscillator. Must be positive (ω > 0).
    n : int
        Quantum number (wavefunction order). Must be n >= 0.

    Returns
    -------
    psi : np.ndarray
        Normalized wavefunction of order 'n', evaluated over the input grid 'x'.

    Pre-condition
    -------------
    - n >= 0. If n < 0, the function triggers a debugging checkpoint and halts execution.
    """
    # Constants set to 1 in atomic units.
    hbar = 1.0
    m = 1.0

    # Components of the analytical solution for stationary states.
    prefactor = 1 / np.sqrt(2**n * factorial(n, exact=False)) * ((m * omega) / (np.pi * hbar))**0.25
    x_coeff = np.sqrt(m * omega / hbar)
    exponential = np.exp(-(m * omega * x**2) / (2 * hbar))

    # Complete wavefunction.
    psi = prefactor * exponential * hermite(x_coeff * x, n)
    return psi

## Initialization for the simulation

Initialize correctly the real space and the momentum grid.
Be careful about the initialization of the momentum, how it should be treated.
Have a look at the `np.fft.fft` documentation.

<details>
  <summary>Solution part 1</summary>

```python
self.x = np.arange(-xmax + xmax / num_x, xmax, self.dx)
...
self.k = np.concatenate((np.arange(0, num_x / 2),
                                 np.arange(-num_x / 2, 0))) * self.dk
```
</details>

In [ ]:
class Param:
    """
    Container for holding all simulation parameters

    Parameters
    ----------
    xmax : float
        The real space is between [-xmax, xmax]
    num_x : int
        Number of intervals in [-xmax, xmax]
    dt : float
        Time discretization
    timesteps : int
        Number of timestep
    im_time : bool, optional
        If True, use imaginary time evolution.
        Default to False.
    """
    def __init__(self,
                 xmax: float,
                 num_x: int,
                 dt: float,
                 timesteps: int,
                 im_time: bool = False) -> None:

        self.xmax = xmax
        self.num_x = num_x
        self.dt = dt
        self.timesteps = timesteps
        self.im_time = im_time

        self.dx = 2 * xmax / num_x      # it's like 2L/N
        # Real time grid
        self.x = np.arange(-xmax + xmax / num_x, xmax, self.dx)
        # Definition of the momentum
        self.dk = np.pi / xmax          # k = pi / L
        # Momentum grid
        self.k = np.concatenate((np.arange(0, num_x / 2), np.arange(-num_x / 2, 0))) * self.dk


class Operators:
    """Container for holding operators and wavefunction coefficients."""
    def __init__(self, res: int) -> None:

        self.V = np.empty(res, dtype=complex)
        self.R = np.empty(res, dtype=complex)
        self.K = np.empty(res, dtype=complex)
        self.wfc = np.empty(res, dtype=complex)

## Set the evolution operators for the imaginary time

Set the coefficiant `coeff` such that it enables real or
imaginary time evolution

<details>
  <summary>Solution part 2</summary>

```python
1 if par.im_time else 1j
```
</details>

In [ ]:
def init(par: Param, omega : float,  voffset: float, wfcoffset: float, n:int = 0) -> Operators:
    """
    Initialize the wavefunction coefficients and the potential.

    Parameters
    ----------
    par: Param
        Class containing the parameters of the simulation
    voffset : float, optional
        Offset of the quadratic potential in real space
        Default to 0.
    wfcoffset: float, optional
        Offset of the wavefunction in real space. Default to 0.
    n : int, optional
        Order of the hermite polynomial (i.e. level of eigenstate)

    Returns
    -------
    Operators
        Filled operators
    """
    opr = Operators(len(par.x))

    # Quadratic potential
    opr.V = 0.5 * (omega**2) * (par.x - voffset) ** 2
    opr.wfc = stationary_state(par.x-wfcoffset, n).astype(complex)

    # Complete here to implement imaginary time evolution based on par.im_time
    coeff = 1 if par.im_time else 1j
    # CREATE THE EXPONENTIAL PART OF THE HAMILTONIAN (we exploit the Hausdorff expansion of the matrix exponential e^{A+B}\sim e^{A}e^{B}+ o(dt^{2}))
    # Operator in momentum space
    opr.K = np.exp(-0.5 * (par.k ** 2) * par.dt * coeff)
    # Operator in real space
    opr.R = np.exp(-0.5 * opr.V * par.dt * coeff)

    return opr


## Time evolution

Complete the split operator method filling the part for imaginary time evolution.
The wavefunction should be normalized at any time!

<details>
  <summary>Solution part 3</summary>

```python
renorm_factor = np.sum( density * par.dx )
opr.wfc /= np.sqrt(renorm_factor)
```
</details>

In [ ]:
def split_op(par: Param, opr: Operators) -> None:
    """
    Split operator method for time evolution.
    Works in place.
    Returns the densities in real and momentum space

    Parameters
    ----------
    par : Param
        Parameters of the simulation
    opr : Operators
        Operators of the simulation

    Returns
    -------
    None
        Acts in place
    """
    # Store the results in res.
    # Use the first 100 to store the wfc in real
    # space and the other half to store the wavefunction
    # in momentum space
    # Initialize res to store real and momentum space densities separately
    res = np.zeros((2, par.timesteps, par.num_x))
    for i in range(par.timesteps):

        # Half-step in real space
        opr.wfc *= opr.R

        # FFT to momentum space
        opr.wfc = np.fft.fft(opr.wfc)

        # Full step in momentum space
        opr.wfc *= opr.K

        # iFFT back
        opr.wfc = np.fft.ifft(opr.wfc)

        # Final half-step in real space
        opr.wfc *= opr.R

        # Density for plotting and potential
        density = np.abs(opr.wfc) ** 2
        density_momentum = np.abs(np.fft.fft(opr.wfc))**2

        # Renormalizing for imaginary time
        if par.im_time:
            # Compute the renormalization factor
            renorm_factor = np.sum( density * par.dx )
            # Renormalize the wavefunction opr.wfc
            opr.wfc /= np.sqrt(renorm_factor)

        # Save to res
        res[0, i, :] = np.real(density)  # Real space
        res[1, i, :] = np.real(density_momentum)  # Momentum space

    return res

In [ ]:
def init_and_splitop(par: Param, omega : float,  voffset: float, wfcoffset: float, n:int = 0) -> Operators:

    opr = Operators(len(par.x))

    # Quadratic potential
    # opr.V = 0.5 * (omega**2) * (par.x - voffset) ** 2
    opr.wfc = stationary_state(par.x-wfcoffset, n).astype(complex)

    # Complete here to implement imaginary time evolution based on par.im_time
    coeff = 1 if par.im_time else 1j
    # CREATE THE EXPONENTIAL PART OF THE HAMILTONIAN (we exploit the Hausdorff expansion of the matrix exponential e^{A+B}\sim e^{A}e^{B}+ o(dt^{2}))
    # Operator in momentum space
    # opr.K = np.exp(-0.5 * (par.k ** 2) * par.dt * coeff)
    # Operator in real space
    # opr.R = np.exp(-0.5 * opr.V * par.dt * coeff)


    res = np.zeros((2, par.timesteps, par.num_x))
    for i in range(par.timesteps):
        # Time-dependent q_0
        q_0 = (i * par.dt) / par.T
        opr.V = 0.5 * omega**2 * (par.x - q_0)**2
        
        opr.R = np.exp(-0.5 * opr.V * par.dt * coeff)
        opr.K = np.exp(-0.5 * (par.k ** 2) * par.dt * coeff)

        # Half-step in real space
        opr.wfc *= opr.R

        # FFT to momentum space
        opr.wfc = np.fft.fft(opr.wfc)

        # Full step in momentum space
        opr.wfc *= opr.K

        # iFFT back
        opr.wfc = np.fft.ifft(opr.wfc)

        # Final half-step in real space
        opr.wfc *= opr.R

        # Density for plotting and potential
        density = np.abs(opr.wfc) ** 2
        density_momentum = np.abs(np.fft.fft(opr.wfc))**2

        # Renormalizing for imaginary time
        if par.im_time:
            # Compute the renormalization factor
            renorm_factor = np.sum( density * par.dx )
            # Renormalize the wavefunction opr.wfc
            opr.wfc /= np.sqrt(renorm_factor)

        # Save to res
        res[0, i, :] = np.real(density)  # Real space
        res[1, i, :] = np.real(density_momentum)  # Momentum space


    return res


In [ ]:
def calculate_energy(par: Param, opr: Operators) -> float:
    """Calculate the energy <Psi|H|Psi>."""
    # Creating real, momentum, and conjugate wavefunctions.
    wfc_r = opr.wfc
    wfc_k = np.fft.fft(wfc_r)
    wfc_c = np.conj(wfc_r)

    # Finding the momentum and real-space energy terms
    energy_k = 0.5 * wfc_c * np.fft.ifft((par.k ** 2) * wfc_k)
    energy_r = wfc_c * opr.V * wfc_r

    # Integrating over all space
    energy_final = sum(energy_k + energy_r).real

    return energy_final * par.dx

## Code 

In [ ]:
N = 1000    # Number of intervals between -L and L
L = 4
dt = 0.025
t_steps = 20000

voffset=0       # Potential Offset --> it's q - q_0


wfcoffset=voffset     # W.f. offset --> it's x - x_0
n=1             # Order of the Hermite Polynomial --> correspond to the ground state


params = Param(xmax=L, num_x=N, dt=dt, timesteps=t_steps, im_time=True)

ops = init(params, voffset, wfcoffset, n)

res = split_op(params, ops)


# I'm computing the custom_wfc as the sum of the first 10 eigenvectors and than I normalize it
custom_wfc = np.zeros(N, dtype=complex)
for n in range(10):     
    custom_wfc += stationary_state(params.x - wfcoffset, n).astype(complex)

print(norm(custom_wfc))

if custom_wfc is not None:
    # First ensure it is normalized
    custom_wfc /= norm(custom_wfc)
    ops.wfc = custom_wfc
    print(norm(custom_wfc))

In [ ]:


print(f"The final energy of your systems is: {calculate_energy(params, ops)}")